In [1]:
import mysql.connector
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [4]:
db_user = os.environ.get('my_username')
db_password = os.environ.get('my_password')
db = mysql.connector.connect(host= "localhost", user = db_user, passwd = db_password, database = "clique_bait")
db

# Queries

# A. Enterprise Relationship Diagram

In [ ]:
# UPDATING

# B. Digital Analysis

In [5]:
# How many users are there

b1=  pd.read_sql_query("select count(distinct user_id) user_count from users",db)
b1 

,user_count
0,500


In [6]:
# How many cookies does each user have on average

b2= pd.read_sql_query("""with cte as (select *, count(user_id) as count_cookies from users group by user_id order by user_id)
select avg(cte.count_cookies) as avg_cookies from cte""",db)
b2

,avg_cookies
0,3.564


In [7]:
# What is the unique number of visits by all users per month?

b3 =pd.read_sql_query("""select DATE_FORMAT(event_time, '%M') month_name, count(distinct visit_id) unique_visits from events
group by DATE_FORMAT(event_time, '%M') order by count(distinct visit_id) asc""",db)
b3

,month_name,unique_visits
0,May,36
1,April,248
2,January,876
3,March,916
4,February,1488


In [8]:
# What is the number of events for each event type?

b4 = pd.read_sql_query("select event_type, count(visit_id) num_visit from events group by event_type",db)
b4

,event_type,num_visit
0,1,20928
1,2,8451
2,3,1777
3,4,876
4,5,702


In [9]:
# What is the percentage of visits which have a purchase event?

b5 = pd.read_sql_query("""select (COUNT(DISTINCT e.visit_id)/(SELECT COUNT(DISTINCT visit_id) FROM clique_bait.events))*100 as percentage
from events e 
inner join event_identifier ei on e.event_type=ei.event_type
where ei.event_name = "purchase" """,db)
b5

,percentage
0,49.8597


In [10]:
# What is the percentage of visits which view the checkout page but do not have a purchase event?

b6 = pd.read_sql_query("""with cte as (select visit_id, 
max(case when event_type = 1 and page_id = 12 then 1 else 0 end) as checkout, 
max(case when event_type = 3 then 1 else 0 end) as purchase 
from events
group by visit_id)""",db)
b6

## UPDATING

DatabaseError: Execution failed on sql 'with cte as (select visit_id, 
max(case when event_type = 1 and page_id = 12 then 1 else 0 end) as checkout, 
max(case when event_type = 3 then 1 else 0 end) as purchase 
from events
group by visit_id)': 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '' at line 5

In [11]:
# What are the top 3 pages by number of views?

b7 = pd.read_sql_query("""select page_id, COUNT(*) AS page_views
from events
group BY page_id
order BY page_views desc
limit 3""",db)
b7

,page_id,page_views
0,2,4752
1,9,2515
2,10,2513


In [12]:
# What is the number of views and cart adds for each product category?

b8 = pd.read_sql_query("""select ph.product_category,
sum(case when e.event_type = 1 then 1 else 0 end) as page_views,
sum(case when e.event_type=2 then 1 else 0 end) as add_cart
from events e 
inner join page_hierarchy ph on e.page_id = ph.page_id
where ph.product_category is not null
group by ph.product_category
order by page_views""",db)
b8

,product_category,page_views,add_cart
0,Luxury,3032.0,1870.0
1,Fish,4633.0,2789.0
2,Shellfish,6204.0,3792.0


In [13]:
# what are the top 3 products by purchases?

b9 =pd.read_sql_query("""select * 
from events e
inner join page_hierarchy ph on e.page_id = ph.page_id
where e.event_type = 3""",db)
b9

,visit_id,cookie_id,page_id,event_type,sequence_number,event_time,page_id,page_name,product_category,product_id
0,ccf365,c4ca42,13,3,11,2020-02-04 19:21:26,13,Confirmation,None,None
1,9a2f24,eccbc8,13,3,9,2020-02-21 03:22:23,13,Confirmation,None,None
2,f61ed7,e4da3b,13,3,11,2020-02-01 06:35:38,13,Confirmation,None,None
3,e0ce49,167909,13,3,13,2020-01-25 22:49:37,13,Confirmation,None,None
4,8479c1,8f14e4,13,3,7,2020-02-09 17:31:44,13,Confirmation,None,None
...,...,...,...,...,...,...,...,...,...,...
1772,b40bf5,603b54,13,3,17,2020-03-01 04:12:14,13,Confirmation,None,None
1773,96b841,704a55,13,3,19,2020-03-14 10:37:00,13,Confirmation,None,None
1774,672c30,c9bfa9,13,3,14,2020-03-14 10:41:37,13,Confirmation,None,None
1775,c09cfd,d345a8,13,3,15,2020-02-23 01:33:52,13,Confirmation,None,None


# C. Project Funnel Analysis

In [14]:
# In a single query statement, create new output table which consists of below column :
# - How many times was each product viewed?
# - How many times was each product added to cart?
# - How many times was each product added to a cart but not purchased (abandoned)?
# - How many times was each product purchased?


c1 = pd.read_sql_query("""select ph.page_name, ph.product_category,
sum(case when e.event_type = 1 then 1 else 0 end) as product_viewed,
sum(case when e.event_type = 2 then 1 else 0 end) as cart_added
from events e
inner join page_hierarchy ph on e.page_id = ph.page_id
where ph.product_category is not null
group by ph.page_name, ph.product_category
order by ph.product_category""",db)
c1

# UPDATINGGGG

,page_name,product_category,product_viewed,cart_added
0,Kingfish,Fish,1559.0,920.0
1,Tuna,Fish,1515.0,931.0
2,Salmon,Fish,1559.0,938.0
3,Russian Caviar,Luxury,1563.0,946.0
4,Black Truffle,Luxury,1469.0,924.0
5,Lobster,Shellfish,1547.0,968.0
6,Crab,Shellfish,1564.0,949.0
7,Oyster,Shellfish,1568.0,943.0
8,Abalone,Shellfish,1525.0,932.0
